---------------------------------------------------------

Here is the entry point for the problem solver.  It works for an arbitrary Wumpus world of size N.  
It is implemented entirely with two helper functions that you will write:  breezeMeansAdjacentPit and visitMeansNoPit
both of which return a set of axioms.  

In [ ]:
#  Need a special check for no activity, because sending an empty
#  string to the theorem prover breaks it

def wumpusQuery(query, activity, worldSize, verbose=False):
    premises = breezeMeansAdjacentPit(worldSize) + visitMeansNoPit()
    if len(activity) > 0:
        premises.append(activity)
    return prove(query, premises, verbose)

This next test demonstrates the problem solver on the size 2 world.  
It shows that
* with no activity, nothing is known about pits
* with a visit to (1,1) and a breeze you can infer the disjunction of (1,2) and (2,1) but not either one singly
* with an additional visit to (1,2) and no breeze you know for sure there's a pit at (2,1) and that there is no pit at either (1,1) or (2,2)

In [ ]:
from nltk.sem import Expression     # Must need to include 
from nltk import ResolutionProver   # Must need to include

#  For all squares, if you have visited a square, you know there is no pit there

def visitMeansNoPit():
    # Return a list of strings (logical sentences)
    p = ["all x.(all y. (visit(x,y) -> -pit(x,y)))"] # for every square
    return p

# For all squares, if you hear a breeze at the square, then there is a pit in at least
# one adjacent square.

def breezeMeansAdjacentPit(n):
    # Return a list of strings (logical sentences)
    p = []

    for i in range(1,n+1):  # for every square within n*n world
        for j in range(1,n+1):
            # boundary check, 1 <= i <= n and 1 <= j <= n    
            adjacentList = list(filter(lambda p: p[0] >= 1 and p[0] <=n and p[1] >= 1 and p[1] <= n , 
                                       [(i+1,j), (i-1,j), (i,j+1), (i,j-1)]))   
            
            # default expression, ex: "(breeze(i, j) ->"
            breezeStr = "(breeze({0}, {1}) ->".format(str(i), str(j))
            
            # adjacent information, ex: " pit(x1,y1) | pit(x2,y2) | pit(x3,y3))", the num of pit(x,y) is varied
            pitAxiom = breezeStr + '|'.join(list(map(lambda p: " pit{0} ".format(p), adjacentList))) + ')'
        
            p.append(pitAxiom)        
    return p

def prove(goal, premises, verbose=False): # Need to have this function because it is used in wumpusQuery
    read_expr = Expression.fromstring
    return ResolutionProver().prove(read_expr(goal), [read_expr(p) for p in premises], verbose=verbose)

In [ ]:
def testQueries(activity, worldSize):
    print("Activity is " + activity)
    print("Pit at either (1,2) or (2,1): " + str(wumpusQuery('pit(1,2) | pit(2,1)', activity, worldSize)))
    print("Pit at (1,1): " + str(wumpusQuery('pit(1,1)', activity, worldSize)))
    print("Pit at (1,2): " + str(wumpusQuery('pit(1,2)', activity, worldSize)))
    print("Pit at (2,1): " + str(wumpusQuery('pit(2,1)', activity, worldSize)))
    print("Pit at (2,2): " + str(wumpusQuery('pit(2,2)', activity, worldSize)))
    print()

activity = ''
testQueries(activity, 2)

activity = 'visit(1,1) ^ breeze(1,1)'
testQueries(activity, 2)

activity = 'visit(1,1) ^ breeze(1,1) ^ visit(1,2) ^ -breeze(1,2)'
testQueries(activity, 2)


<pre>
Activity is 
Pit at either (1,2) or (2,1): False
Pit at (1,1): False
Pit at (1,2): False
Pit at (2,1): False
Pit at (2,2): False

Activity is visit(1,1) ^ breeze(1,1)
Pit at either (1,2) or (2,1): True
Pit at (1,1): False
Pit at (1,2): False
Pit at (2,1): False
Pit at (2,2): False

Activity is visit(1,1) ^ breeze(1,1) ^ visit(1,2) ^ -breeze(1,2)
Pit at either (1,2) or (2,1): True
Pit at (1,1): False
Pit at (1,2): False
Pit at (2,1): True
Pit at (2,2): False
</pre>

Here you need to define the two helper functions that generate axioms that will be fed into the 
theorem prover via the wumpusQuery function